# Init

In [2]:
cd ..

/Users/wliao0504/code/clif/CLIF-MIMIC


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
pwd

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [4]:
# src/tables/patient_assessments.py
import numpy as np
import pandas as pd
import logging
import duckdb
from importlib import reload
import src.utils as utils
reload(utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, con, REPO_ROOT, mimic_table_pathfinder, \
    search_mimic_items

loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json
loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


# Dev

In [8]:
dialysis_mapping = load_mapping_csv("dialysis")
dialysis_mapping

,clif_variable,table,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,224146,System Integrity,System Integrity,chartevents,Dialysis,NaN,Text,241835,"{'Active': 176983, 'Clots Present': 43537, 'Ne...",NaN,NaN,NaN,NaN
1,ultrafiltration_out,crrt_therapy,226457,Ultrafiltrate Output,Ultrafiltrate Output,chartevents,Dialysis,mL,Numeric,217653,"Max: 16800.0, Min: -600.0, Mean: 365.13",NaN,NaN,this acounts for rr in addition to dr,should be highest for H > DF > D
2,NaN,NaN,224191,Hourly Patient Fluid Removal,Hourly PFR,chartevents,Dialysis,mL,Numeric,209426,"Max: 150179.0, Min: -950.0, Mean: 374.84",NaN,NaN,NaN,HDF > HD > H
3,NaN,NaN,224149,Access Pressure,Access Pressure,chartevents,Dialysis,mmHg,Numeric,206268,"Max: 500.0, Min: -308135.0, Mean: -52.29",NaN,NaN,NaN,NaN
4,NaN,NaN,224150,Filter Pressure,Filter Pressure,chartevents,Dialysis,mmHg,Numeric,206263,"Max: 60132.0, Min: -503.0, Mean: 122.07",NaN,NaN,NaN,NaN
5,NaN,NaN,225183,Current Goal,Current Goal,chartevents,Dialysis,mL,Numeric,206147,"Max: 10000.0, Min: -15010.0, Mean: -43.13",NaN,NaN,NaN,NaN
6,NaN,NaN,224151,Effluent Pressure,Effluent Pressure,chartevents,Dialysis,mmHg,Numeric,206103,"Max: 5757.0, Min: -2735.0, Mean: -15.05",NaN,NaN,NaN,NaN
7,NaN,NaN,224152,Return Pressure,Return Pressure,chartevents,Dialysis,mmHg,Numeric,206077,"Max: 5747.0, Min: -329.0, Mean: 55.53",NaN,NaN,NaN,NaN
8,dialysate_flow_rate,crrt_therapy,224154,Dialysate Rate,Dialysate Rate,chartevents,Dialysis,ml/hr,Numeric,200611,"Max: 1400000.0, Min: 0.0, Mean: 743.96",NaN,NaN,NaN,NaN
9,NaN,NaN,224153,Replacement Rate,Replacement Rate,chartevents,Dialysis,ml/hr,Numeric,199530,"Max: 381600.0, Min: -1800.0, Mean: 1712.85",NaN,NaN,NaN,NaN


In [10]:
crrt_items = dialysis_mapping[dialysis_mapping["table"].astype(str).str.contains("crrt")]
crrt_items

,clif_variable,table,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
1,ultrafiltration_out,crrt_therapy,226457,Ultrafiltrate Output,Ultrafiltrate Output,chartevents,Dialysis,mL,Numeric,217653,"Max: 16800.0, Min: -600.0, Mean: 365.13",NaN,NaN,this acounts for rr in addition to dr,should be highest for H > DF > D
8,dialysate_flow_rate,crrt_therapy,224154,Dialysate Rate,Dialysate Rate,chartevents,Dialysis,ml/hr,Numeric,200611,"Max: 1400000.0, Min: 0.0, Mean: 743.96",NaN,NaN,NaN,NaN
12,blood_flow_rate,crrt_therapy,224144,Blood Flow (ml/min),Blood Flow (ml/min),chartevents,Dialysis,ml/min,Numeric,196378,"Max: 1801300.0, Min: -120.0, Mean: 167.79",NaN,NaN,NaN,NaN
13,pre_filter_replacement_fluid_rate,crrt_therapy,228005,PBP (Prefilter) Replacement Rate,PBP (Prefilter) Replacement Rate,chartevents,Dialysis,ml/hr,Numeric,183428,"Max: 1822540.0, Min: -601600.0, Mean: 1590.45",NaN,NaN,NaN,NaN
14,post_filter_replacement_fluid_rate,crrt_therapy,228006,Post Filter Replacement Rate,Post Filter Replacement Rate,chartevents,Dialysis,ml/hr,Numeric,179745,"Max: 5000.0, Min: -35300.0, Mean: 232.86",NaN,NaN,NaN,NaN
18,crrt_mode_name,crrt_therapy,227290,CRRT mode,CRRT mode,chartevents,Dialysis,NaN,Text,121114,"{'CVVHDF': 116065, 'CVVHD': 2874, 'CVVH': 2113...",NaN,NaN,NaN,HDF: both rate


In [15]:
crrt_id_to_variable_mapper = dict(zip(crrt_items["itemid"], crrt_items["clif_variable"]))
crrt_id_to_variable_mapper

{226457: 'ultrafiltration_out',
 224154: 'dialysate_flow_rate',
 224144: 'blood_flow_rate',
 228005: 'pre_filter_replacement_fluid_rate',
 228006: 'post_filter_replacement_fluid_rate',
 227290: 'crrt_mode_name'}

In [11]:
crrt_item_ids = crrt_items["itemid"].tolist()
crrt_item_ids

[226457, 224154, 224144, 228005, 228006, 227290]

In [12]:
crrt_events = fetch_mimic_events(crrt_item_ids)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
crrt_events.head()

,itemid,label,hadm_id,stay_id,time,value,valueuom
0,224154,Dialysate Rate,28317408,32824762,2150-05-21 07:00:00,800,ml/hr
1,228005,PBP (Prefilter) Replacement Rate,28317408,32824762,2150-05-21 07:00:00,1400,ml/hr
2,228006,Post Filter Replacement Rate,28317408,32824762,2150-05-21 07:00:00,200,ml/hr
3,224154,Dialysate Rate,28317408,32824762,2150-05-21 08:00:00,800,ml/hr
4,228005,PBP (Prefilter) Replacement Rate,28317408,32824762,2150-05-21 08:00:00,1400,ml/hr
